# Macro Examples
© 2025 Jim Domeij & Marek Ozana

These examples show how to use `polars-bloomberg` for macro analysis, inspired by ideas from my friend **Jim Domeij**.  
Thank you, Jim, for contributing your perspective and making it possible for others to learn from your macro thinking.

In [1]:
import altair as alt
import polars as pl

from polars_bloomberg import BQuery

## Visualizing S&P 500 Valuation Scenarios

This chart shows potential S&P 500 price levels and returns for the next 12 months. It uses a scenario matrix driven by two factors: Forward P/E ratios (vertical axis, from historical averages) and Forward EPS growth (horizontal axis, centered on consensus). Each cell displays the resulting index price and percentage return from the current price. This helps investors visualize risk and potential upside under different market conditions.


In [2]:
query = """
    let(
        #last_price = px_last();
        #12m_trail_eps = headline_eps_market(fpt=LTM);
        #12m_fwd_eps = headline_eps_market(fpt=BT, fpo=1);
        #epsg_consensus = #12m_fwd_eps/#12m_trail_eps - 1;
        #current_pe = headline_pe_ratio(fpt=BT, fpo=1);
        #5y_avg_pe = avg(headline_pe_ratio(fpt=BT, fpo=1, fill=PREV, dates=range(-5Y, 0D, frq=W)));
        #10y_avg_pe = avg(headline_pe_ratio(fpt=BT, fpo=1, fill=PREV, dates=range(-10Y, 0D, frq=W)));
        #20y_avg_pe = avg(headline_pe_ratio(fpt=BT, fpo=1, fill=PREV, dates=range(-20Y, 0D, frq=W)));
        #35y_avg_pe = avg(headline_pe_ratio(fpt=BT, fpo=1, fill=PREV, dates=range(-35Y, 0D, frq=W)));
        #max_pe = max(headline_pe_ratio(fpt=BT, fpo=1, fill=PREV, dates=range(-35Y, 0D, frq=W)));
    )
    get(
        #last_price,
        #12m_trail_eps,
        #12m_fwd_eps,
        #epsg_consensus,
        #current_pe,
        #5y_avg_pe,
        #10y_avg_pe,
        #20y_avg_pe,
        #35y_avg_pe,
        #max_pe
    )
    preferences(dropCols=['CURRENCY', 'DATE', 'AS_OF_DATE', 'REVISION_DATE', 'PERIOD_END_DATE'])
    for('SPX Index')
"""  # noqa: E501
with BQuery() as bq:
    res = bq.bql(query)

df = res.combine()
df

ID,#last_price,#12m_trail_eps,#12m_fwd_eps,#epsg_consensus,#current_pe,#5y_avg_pe,#10y_avg_pe,#20y_avg_pe,#35y_avg_pe,#max_pe
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""SPX Index""",6833.33,269.207986,307.10891,0.140787,22.231234,20.040917,18.823455,16.3336,16.773689,25.312711


In [3]:
# Prepare data for plotting and create chart
GROWTH_OFFSETS = [-0.06, -0.04, -0.02, 0.0, 0.02, 0.04, 0.06]


def calculate_scenarios(df: pl.DataFrame) -> pl.DataFrame:
    """Calculate price and return scenarios."""
    # Extract scalar values from the single-row result

    data = df.row(0, named=True)

    last_price = data["last_price"]
    trailing_eps = data["12m_trail_eps"]
    consensus_growth = data["epsg_consensus"]

    # Define P/E scenarios based on historical averages and extremes
    pe_scenarios = {
        data["max_pe"]: "highest",
        data["current_pe"]: "current",
        data["5y_avg_pe"]: "5y avg",
        data["10y_avg_pe"]: "10y avg",
        data["20y_avg_pe"]: "20y avg",
        data["35y_avg_pe"]: "35y avg",
    }

    # Sort by P/E value for the Y-axis
    sorted_pe_values = sorted(pe_scenarios.keys())

    # Create mappings dataframe for robust joining
    pe_labels_df = pl.DataFrame(
        {
            "fwd_pe": list(pe_scenarios.keys()),
            "pe_label_text": list(pe_scenarios.values()),
        }
    )

    # Define EPS growth scenarios centered around consensus
    growth_scenarios = [consensus_growth + x for x in GROWTH_OFFSETS]

    # Create the scenarios dataframe
    scenarios = (
        pl.DataFrame({"fwd_pe": sorted_pe_values})
        .join(pl.DataFrame({"fwd_eps_growth": growth_scenarios}), how="cross")
        .join(pe_labels_df, on="fwd_pe", how="left")
        .with_columns(
            fwd_eps=pl.lit(trailing_eps) * (1 + pl.col("fwd_eps_growth")),
            price=pl.col("fwd_pe")
            * pl.lit(trailing_eps)
            * (1 + pl.col("fwd_eps_growth")),
            # Mark the "current" scenario for highlighting
            is_current=(pl.col("fwd_pe") == data["current_pe"])
            & (pl.col("fwd_eps_growth") == consensus_growth),
        )
        .with_columns(return_pct=(pl.col("price") / pl.lit(last_price) - 1) * 100)
        .with_columns(
            # Create display labels
            price_label=pl.col("price").round(0).cast(pl.Int64).cast(pl.Utf8),
            return_label=pl.format(
                "({}%)", pl.col("return_pct").round(0).cast(pl.Int64)
            ),
        )
        .with_columns(
            pe_display=pl.format(
                "{} {}", pl.col("fwd_pe").round(1), pl.col("pe_label_text")
            )
        )
    )
    return scenarios  # noqa: RET504


def create_chart(scenarios: pl.DataFrame) -> alt.Chart:
    """Create an Altair heatmap chart from the scenarios dataframe."""
    # Base chart definition
    base = alt.Chart(scenarios).encode(
        x=alt.X(
            "fwd_eps_growth:O",
            title="12m forward EPS growth",
            axis=alt.Axis(format="+.1%"),
            sort="ascending",
        ),
        y=alt.Y(
            "pe_display:O",
            title="12m fwd P/E",
            sort=alt.SortField(field="fwd_pe", order="descending"),
        ),
    )

    # Heatmap rectangles colored by return
    rect = base.mark_rect(opacity=0.7).encode(
        color=alt.Color(
            "return_pct:Q",
            title="Price return",
        )
        .scale(scheme="blueorange", reverse=True)
        .legend(None),
        tooltip=[
            alt.Tooltip("fwd_pe:Q", format=".1f", title="Fwd P/E"),
            alt.Tooltip("fwd_eps:Q", format=".2f", title="Fwd EPS"),
            alt.Tooltip("price:Q", format=".0f", title="Price target"),
            alt.Tooltip("return_pct:Q", format="+.1f", title="Return (%)"),
        ],
    )

    # Highlight the current scenario with a thick border
    highlight = base.transform_filter(alt.datum.is_current).mark_rect(
        fill=None, stroke="black", strokeWidth=3
    )

    # Text labels
    text_price = base.mark_text(baseline="middle", fontWeight="bold", dy=-8).encode(
        text="price_label:N"
    )

    text_return = base.mark_text(baseline="middle", dy=8, size=10).encode(
        text="return_label:N",
        color=alt.value("black"),
    )

    chart = (rect + highlight + text_price + text_return).properties(
        width=650,
        height=360,
        title="S&P 500 12-month price level (return) based on EPS and P/E",
    )
    return chart  # noqa: RET504


# Clean column names (remove '#' prefix)
df = df.rename({col: col.lstrip("#") for col in df.columns})

# Calculate scenarios and create chart
scenarios = calculate_scenarios(df)
chart = create_chart(scenarios)
chart.show()


alt.LayerChart(...)

## S&P 500 Earning Revisions Breadth

This calculates, for each GICS sector within the S&P 500, the number of companies with upward or downward EPS revisions over the past month. It can be used to track the breadth of change in analysts’ EPS estimates by sector.

In [4]:
# Get counts of comps with positive and negative earnings revisions over the past month
query = """
    let(
        #rev = net_chg(is_eps(fpt=BT, fpo=1, ae=e, dates=range(-1M, 0d)));
        #sector = classification_name(gics, 1);

        #up   = sum(group(if(#rev > 0, 1, 0), #sector));
        #down = sum(group(if(#rev < 0, 1, 0), #sector));
    )
    get(#up, #down)
    for(members("SPX Index"))
    preferences(dropCols=["ORIG_IDS", "#sector"])
    """

with BQuery() as bq:
    df = bq.bql(query).combine()
print(df.head())

shape: (5, 3)
┌────────────────────────┬──────┬───────┐
│ ID                     ┆ #up  ┆ #down │
│ ---                    ┆ ---  ┆ ---   │
│ str                    ┆ f64  ┆ f64   │
╞════════════════════════╪══════╪═══════╡
│ Communication Services ┆ 18.0 ┆ 3.0   │
│ Consumer Discretionary ┆ 41.0 ┆ 8.0   │
│ Consumer Staples       ┆ 29.0 ┆ 7.0   │
│ Energy                 ┆ 10.0 ┆ 12.0  │
│ Financials             ┆ 66.0 ┆ 8.0   │
└────────────────────────┴──────┴───────┘


In [5]:
# Chart the results
g_data = df.select(
    pl.col("ID"),
    pl.col("#up").alias("up"),
    (-pl.col("#down")).alias("down"),
    (pl.col("#up") - pl.col("#down")).alias("net"),
)

base = alt.Chart(g_data, title="S&P 500 Earnings Revisions Breadth").encode(
    y=alt.Y("ID:N").title(None)
)

bars = (
    base.transform_fold(["up", "down"])
    .mark_bar()
    .encode(
        x=alt.X("value:Q").title("# of revisions last month"),
        color=alt.Color("key:N")
        .title(None)
        .sort(["up", "down"])
        .legend(orient="bottom"),
    )
)

net_tick = base.mark_tick(color="black", thickness=2, size=18).encode(
    x=alt.X("net:Q"),
)

(bars + net_tick).properties(width=600, height=400)

alt.LayerChart(...)